# Download season wise data for one year

In [1]:
#imports
import geemap
import ee 
import os

In [ ]:
# Create an interactive map to view the results.
Map = geemap.Map(center=[40,100], zoom=4)
Map

In [ ]:
# Get the dataset from the earth engine server.
dataset_l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_RT_TOA')

In [ ]:
# add the necessary shapefile
# This can be a shapefile of the region of interest we want to extract from the whole map. We can also specify some
# geometry objects in the form of polygon for getting the roi, if not available freely.
city_file = './shapefiles/Bengaluru-shapefile/BBMP_Boundary.shp'
roi = geemap.shp_to_ee(city_file)
Map.addLayer(roi, {}, 'Bengaluru')

In [ ]:
# Initialize the parameters
seasons = {'mam':[1,3,31,5],'jjas':[1,6,30,9],'on':[1,10,30,11],'djf':[1,12,28,2],'ann':[1,3,28,2]}
year = 2000
save_path = '../images/'

## Helper Functions

In [ ]:
# clip to shape
def clip_to_shape(image):
    return image.clip(roi)

In [ ]:
# Mask the clouds
def maskclouds(image):
    # Select the BQA band, which has the cloud information
    qa = image.select('BQA')
​
    # In BQA Band bit 4 is the cloud band, hence activate only that bit, set all other bits of BQA to zero
    cloudBit = 1 << 4 #(00000001 -> 00010000) 
​
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBit).eq(0) # And with all the other bits of BQA to check for clouds
    return image.updateMask(mask)

In [ ]:
# Function to define the dates for the specified study interval
def filter_date(year,i,flag):
    start_date = ee.Date.fromYMD(year,seasons[i][1],seasons[i][0])
    if flag==0:
        end_date = ee.Date.fromYMD(year,seasons[i][3],seasons[i][2])
    else:
        end_date = ee.Date.fromYMD(year+1,seasons[i][3],seasons[i][2])
    
    img = (dataset_l7.filterDate(start_date,end_date)
                    .filterBounds(roi)
                    .select('B3','B4')
                    .mean())
    return img

In [ ]:
# Function to loop over the different study intervals
def yearlyMap(year):
    img = filter_date(year,'mam',flag=0)
    filename = os.path.join(save_path,'test.tif')
    filename_ = save_path
    download(img, filename)
    for k,i in enumerate(seasons):
        if i=='djf' or i== 'ann':
            img = filter_date(year,i,flag=1) 
        else:
            img = filter_date(year,i,flag=0)
        filename = os.path.join(save_path,i+'-'+ str(k)+'.tif')
        download(img, filename)

In [ ]:
def download(img,filename):
    geemap.ee_export_image(
    img, 
    filename = filename, 
    scale=None, 
    crs=None, 
    region=roi.geometry(), 
    file_per_band=False, 
    timeout=300, 
    proxies=None
)

## Generate the maps for a particular year

In [ ]:
yearlyMap(2000)